In [1]:
from behavior_and_time_alignment import beh_and_time_alignment
from session_preprocessing import ephys_opto_preprocessing
from opto_tagging import opto_plotting_session, opto_plotting_unit
from opto_waveforms_preprocessing import opto_wf_preprocessing
import matplotlib.pyplot as plt
from utils.beh_functions import session_dirs
import numpy as np
import pandas as pd
import os

In [2]:
session_list = ['behavior_758017_2025-02-04_11-57-38',
                'behavior_758017_2025-02-05_11-42-34',
                'behavior_758017_2025-02-06_11-26-14',
                'behavior_758017_2025-02-07_14-11-08']

In [2]:
session_list = ['behavior_751004_2024-12-19_11-50-37',
'behavior_751004_2024-12-20_13-26-11',
'behavior_751004_2024-12-21_13-28-28',
'behavior_751004_2024-12-22_13-09-17',
'behavior_751004_2024-12-23_14-20-03']


In [2]:
session_assets = pd.read_csv('/root/capsule/code/data_management/session_assets.csv')
session_list = session_assets['session_id']

In [3]:
aniIDs = ['751766', '751769']
session_list = [session for session in session_list if any(aniID in session for aniID in aniIDs)]
session_list

['behavior_751769_2025-01-16_11-32-05',
 'behavior_751769_2025-01-17_11-37-39',
 'behavior_751769_2025-01-18_10-15-25',
 'behavior_751766_2025-02-11_11-53-38',
 'behavior_751766_2025-02-13_11-31-21',
 'behavior_751766_2025-02-14_11-37-11',
 'behavior_751766_2025-02-15_12-08-11']

In [4]:
for session in session_list[:-1]:
    # beh_and_time_alignment(session)
    plt.close('all')
    ephys_opto_preprocessing(session, 'raw', 'soma')
    plt.close('all')
    opto_wf_preprocessing(session, 'raw', 'soma', load_sorting_analyzer=True)
    plt.close('all')
    opto_plotting_session(session, 'raw', 'soma', plot=True)
    

behavior_751769_2025-01-16_11-32-05


/root/capsule/code/beh_ephys_analysis/opto_tagging.py:234: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  unit_qc = unit_qc.apply(pd.to_numeric, errors='ignore')


126 out of 233 units pass quality control
{'samples_to_keep': [-30, 60], 'y_neighbors_to_keep': 3, 'orginal_loc': False}


/root/capsule/code/beh_ephys_analysis/opto_tagging.py:269: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  opto_wf = opto_wf.apply(pd.to_numeric, errors='ignore')
/root/capsule/code/beh_ephys_analysis/opto_tagging.py:79: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  opto_wf = opto_wf.apply(pd.to_numeric, errors='ignore')
/root/capsule/code/beh_ephys_analysis/opto_tagging.py:293: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  target_qc = pd.concat([target_qc, pd.DataFrame([qc_dict])], ignore_index=True)
/root/capsu

Merging file 0 out of 233
Merging file 50 out of 233
Merging file 100 out of 233
Merging file 150 out of 233
Merging file 200 out of 233
PDF files in '/root/capsule/scratch/751769/behavior_751769_2025-01-16_11-32-05/ephys/opto/raw/figures' merged into '/root/capsule/scratch/751769/behavior_751769_2025-01-16_11-32-05/ephys/opto/raw/behavior_751769_2025-01-16_11-32-05_opto_tagging.pdf' successfully.
12 out of 233 units pass quality control and opto tagging
behavior_751769_2025-01-17_11-37-39


...

/root/capsule/code/beh_ephys_analysis/opto_tagging.py:234: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  unit_qc = unit_qc.apply(pd.to_numeric, errors='ignore')


125 out of 214 units pass quality control
{'samples_to_keep': [-30, 60], 'y_neighbors_to_keep': 3, 'orginal_loc': False}


/root/capsule/code/beh_ephys_analysis/opto_tagging.py:269: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  opto_wf = opto_wf.apply(pd.to_numeric, errors='ignore')
/root/capsule/code/beh_ephys_analysis/opto_tagging.py:79: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  opto_wf = opto_wf.apply(pd.to_numeric, errors='ignore')
/root/capsule/code/beh_ephys_analysis/opto_tagging.py:293: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  target_qc = pd.concat([target_qc, pd.DataFrame([qc_dict])], ignore_index=True)
/root/capsu

Merging file 0 out of 214
Merging file 50 out of 214
Merging file 100 out of 214
Merging file 150 out of 214
Merging file 200 out of 214


PDF files in '/root/capsule/scratch/751766/behavior_751766_2025-02-14_11-37-11/ephys/opto/raw/figures' merged into '/root/capsule/scratch/751766/behavior_751766_2025-02-14_11-37-11/ephys/opto/raw/behavior_751766_2025-02-14_11-37-11_opto_tagging.pdf' successfully.
25 out of 214 units pass quality control and opto tagging


In [ ]:
for session in session_list:
    ephys_opto_preprocessing(session, 'curated', 'soma')
    opto_wf_preprocessing(session, 'curated', 'soma', load_sorting_analyzer=False)
    opto_plotting_session(session, 'curated', plot=True)
    plt.close('all')

In [6]:
session = 'behavior_751004_2024-12-20_13-26-11'
data_type = 'raw'
target = 'soma'
target_unit_ids = [16]
# pass_df = opto_plotting_session(session, data_type, target, plot=True,  resp_thresh=0.5, lat_thresh=0.015, target_unit_ids=target_unit_ids)

In [3]:
data_type = 'raw'
target = 'soma'
for session in session_list:
    combine_qc = opto_plotting_session(session, data_type, target, plot=True,  resp_thresh=0.5, lat_thresh=0.015)

/root/capsule/code/beh_ephys_analysis/opto_tagging.py:234: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  unit_qc = unit_qc.apply(pd.to_numeric, errors='ignore')


96 out of 271 units pass quality control
{'samples_to_keep': [-30, 60], 'y_neighbors_to_keep': 3, 'orginal_loc': False}


/root/capsule/code/beh_ephys_analysis/opto_tagging.py:269: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  opto_wf = opto_wf.apply(pd.to_numeric, errors='ignore')
/root/capsule/code/beh_ephys_analysis/opto_tagging.py:79: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  opto_wf = opto_wf.apply(pd.to_numeric, errors='ignore')
/root/capsule/code/beh_ephys_analysis/opto_tagging.py:293: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  target_qc = pd.concat([target_qc, pd.DataFrame([qc_dict])], ignore_index=True)
/root/capsu

Merging file 0 out of 271
Merging file 50 out of 271
Merging file 100 out of 271
Merging file 150 out of 271
Merging file 200 out of 271
Merging file 250 out of 271
PDF files in '/root/capsule/scratch/751004/behavior_751004_2024-12-19_11-50-37/ephys/opto/raw/figures' merged into '/root/capsule/scratch/751004/behavior_751004_2024-12-19_11-50-37/ephys/opto/raw/behavior_751004_2024-12-19_11-50-37_opto_tagging.pdf' successfully.
18 out of 271 units pass quality control and opto tagging


/root/capsule/code/beh_ephys_analysis/opto_tagging.py:234: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  unit_qc = unit_qc.apply(pd.to_numeric, errors='ignore')
/root/capsule/code/beh_ephys_analysis/opto_tagging.py:269: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  opto_wf = opto_wf.apply(pd.to_numeric, errors='ignore')
/root/capsule/code/beh_ephys_analysis/opto_tagging.py:79: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  opto_wf = opto_wf.apply(pd.to_numeric, errors='ignore')


...

Merging file 0 out of 114
Merging file 50 out of 114
Merging file 100 out of 114
PDF files in '/root/capsule/scratch/751004/behavior_751004_2024-12-22_13-09-17/ephys/opto/raw/figures' merged into '/root/capsule/scratch/751004/behavior_751004_2024-12-22_13-09-17/ephys/opto/raw/behavior_751004_2024-12-22_13-09-17_opto_tagging.pdf' successfully.
4 out of 114 units pass quality control and opto tagging


/root/capsule/code/beh_ephys_analysis/opto_tagging.py:234: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  unit_qc = unit_qc.apply(pd.to_numeric, errors='ignore')


81 out of 189 units pass quality control
{'samples_to_keep': [-30, 60], 'y_neighbors_to_keep': 3, 'orginal_loc': False}


/root/capsule/code/beh_ephys_analysis/opto_tagging.py:269: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  opto_wf = opto_wf.apply(pd.to_numeric, errors='ignore')
/root/capsule/code/beh_ephys_analysis/opto_tagging.py:79: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  opto_wf = opto_wf.apply(pd.to_numeric, errors='ignore')
/root/capsule/code/beh_ephys_analysis/opto_tagging.py:293: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  target_qc = pd.concat([target_qc, pd.DataFrame([qc_dict])], ignore_index=True)
/root/capsu

Merging file 0 out of 189
Merging file 50 out of 189
Merging file 100 out of 189
Merging file 150 out of 189
PDF files in '/root/capsule/scratch/751004/behavior_751004_2024-12-23_14-20-03/ephys/opto/raw/figures' merged into '/root/capsule/scratch/751004/behavior_751004_2024-12-23_14-20-03/ephys/opto/raw/behavior_751004_2024-12-23_14-20-03_opto_tagging.pdf' successfully.
28 out of 189 units pass quality control and opto tagging


In [13]:
session_dir = session_dirs(session)
opto_wf_csv = os.path.join(session_dir[f'opto_dir_{data_type}'], f'{session}_opto_waveform_metrics.csv')
opto_wf = pd.read_csv(opto_wf_csv)

In [15]:
curr_unit_id = 1
curr_site = 'surface_LC'
curr_power = 50.0
curr_duration = 4
curr_pre_post = 'post'


In [16]:
opto_wf.query('unit_id == @curr_unit_id and site == @curr_site and power == @curr_power and duration == @curr_duration and pre_post == @curr_pre_post')

,unit_id,power,site,duration,pre_post,template,peak_channel,peak_waveform,spont,count,euclidean_norm,correlation
638,1,50.0,surface_LC,4.0,post,[[ 6.80062532 5.70374966 5.55750036 ... ...,CH5,[ -5.484375 1.5356245 8.70187473 -2.41...,0,8,0.053252,0.898952


In [6]:
session = 'behavior_751766_2025-02-15_12-08-11'
beh_and_time_alignment(session)

In [7]:
pip install ipykernel